In [4]:
import pandas as pd
df = pd.read_csv('../feature_data/HappyGo_NOV_DEC_0509.csv', dtype={'UID':str, 'ZIP_Code':str})

In [5]:
df.head()

,UID,Gender_Code,ZIP_Code,AD_TAG,INV_total_amount,INV_avg_amount,INV_count,INV_seller_count,INV_industry_count,INV_industry_per_catering,...,TXN_industry_per_financial,TXN_industry_per_digit,TXN_industry_per_transportation,TXN_industry_per_leisure,TXN_industry_per_health,TXN_industry_per_professional,TXN_industry_per_food,TXN_time_mode,TXN_zip_code,TXN_time_avg_period
0,7538002,M,104,P05,24313.0,245.585859,99.0,27.0,3.0,0.454545,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,23,104,1.115110e+05
1,3381995,F,110,P04,24118.0,618.410256,39.0,22.0,6.0,0.205128,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,23,NaN,6.958009e+05
2,5941571,F,204,P05,100999.0,1311.675325,77.0,37.0,6.0,0.025974,...,0.00,0.3,0.0,0.0,0.0,0.0,0.0,['00' '23'],114,4.558960e+05
3,3168811,F,973,P01,64335.0,2382.777778,27.0,12.0,4.0,0.481481,...,0.14,0.0,0.0,0.0,0.0,0.0,0.0,23,970,4.813280e+05
4,661344,M,236,P06,12085.0,145.602410,83.0,23.0,4.0,0.012048,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,23,NaN,1.987200e+06


In [6]:
cat_list = ['Gender_Code', 'ZIP_Code', 'AD_TAG']
cat_df = df[cat_list].dropna()

from sklearn import preprocessing
def label_encoder(col):
    '''
    label encoder to do SelectKBest method
    '''
    le = preprocessing.LabelEncoder()
    le.fit(cat_df[col])
    new_col = le.transform(cat_df[col])
    return new_col

for col in cat_list:
    cat_df[col] = label_encoder(col)

In [7]:
from sklearn.feature_selection import SelectKBest, chi2
def get_corr_col(X, y):
    '''
    find the highest corr col with target column
    '''
    selK = SelectKBest(chi2, k=1).fit(X, y)
    highest_corr_col = selK.get_feature_names_out(input_features=None)[0]
    return highest_corr_col

def fill_na(df, target_col, most_corr_col):
    '''

    '''
    mask = df[most_corr_col].notna()
    most_corr_col_types = df.loc[mask, most_corr_col].unique()
    df[f'filled_{target_col}'] = df[target_col]
    for type in most_corr_col_types:
        try:
            most_pop = df.loc[df[most_corr_col] == type, target_col].mode()[0]
        except:
            most_pop = df.loc[df[target_col].notna(), target_col].mode()[0]
        df.loc[(df[most_corr_col] == type) & (df[target_col].isna()), f'filled_{target_col}'] = most_pop
    return df

In [8]:
# find the highest corr col with target column
AD_TAG_highest_corr_col = get_corr_col(cat_df[['Gender_Code', 'ZIP_Code']], cat_df['AD_TAG'])
Gender_Code_highest_corr_col = get_corr_col(cat_df[['AD_TAG', 'ZIP_Code']], cat_df['Gender_Code'])
ZIP_Code_highest_corr_col = get_corr_col(cat_df[['AD_TAG', 'Gender_Code']], cat_df['ZIP_Code'])
corr_dic = {'AD_TAG': AD_TAG_highest_corr_col,
     'Gender_Code': Gender_Code_highest_corr_col,
     'ZIP_Code': ZIP_Code_highest_corr_col}

# fillna value
for key, value in corr_dic.items():
     df = fill_na(df, key, value)

In [10]:
df = df.drop(columns=['Gender_Code', 'ZIP_Code', 'AD_TAG'])
df = df.rename(columns={'filled_AD_TAG': 'AD_TAG', 'filled_Gender_Code': 'Gender_Code', 'filled_ZIP_Code': 'ZIP_Code'})

In [12]:
df.columns

Index(['UID', 'INV_total_amount', 'INV_avg_amount', 'INV_count',
       'INV_seller_count', 'INV_industry_count', 'INV_industry_per_catering',
       'INV_industry_per_entertainment',
       'INV_industry_per_financial_insurance',
       'INV_industry_per_health_and_beauty', 'INV_industry_per_manufacture',
       'INV_industry_per_other', 'INV_industry_per_professional_services',
       'INV_industry_per_retail_service', 'INV_industry_per_transportation',
       'INV_industry_per_wholesale', 'INV_time_mode', 'INV_time_avg_period',
       'APP_participate_count', 'APP_bnnr_count', 'APP_game_count',
       'APP_bnnr_per', 'APP_game_per', 'APP_time_mode', 'APP_time_avg_period',
       'TXN_total_count', 'TXN_earn_count', 'TXN_redeem_count',
       'TXN_earn_count_per', 'TXN_redeem_count_per', 'TXN_total_point',
       'TXN_earn_point', 'TXN_redeem_point', 'TXN_earn_point_per',
       'TXN_redeem_point_per', 'TXN_industry_count', 'TXN_industry_per_retail',
       'TXN_industry_per_other', 

In [13]:
df.loc[:, ['INV_total_amount', 'INV_avg_amount', 'INV_count',
       'INV_seller_count', 'INV_industry_count', 'INV_industry_per_catering',
       'INV_industry_per_entertainment',
       'INV_industry_per_financial_insurance',
       'INV_industry_per_health_and_beauty', 'INV_industry_per_manufacture',
       'INV_industry_per_other', 'INV_industry_per_professional_services',
       'INV_industry_per_retail_service', 'INV_industry_per_transportation',
       'INV_industry_per_wholesale',
       'APP_participate_count', 'APP_bnnr_count', 'APP_game_count',
       'APP_bnnr_per', 'APP_game_per',
       'TXN_total_count', 'TXN_earn_count', 'TXN_redeem_count',
       'TXN_earn_count_per', 'TXN_redeem_count_per', 'TXN_total_point',
       'TXN_earn_point', 'TXN_redeem_point', 'TXN_earn_point_per',
       'TXN_redeem_point_per', 'TXN_industry_count', 'TXN_industry_per_retail',
       'TXN_industry_per_other', 'TXN_industry_per_dingding',
       'TXN_industry_per_financial', 'TXN_industry_per_digit',
       'TXN_industry_per_transportation', 'TXN_industry_per_leisure',
       'TXN_industry_per_health', 'TXN_industry_per_professional',
       'TXN_industry_per_food', 'AD_TAG', 'Gender_Code', 'ZIP_Code']] = df.loc[:, ['INV_total_amount', 'INV_avg_amount', 'INV_count',
       'INV_seller_count', 'INV_industry_count', 'INV_industry_per_catering',
       'INV_industry_per_entertainment',
       'INV_industry_per_financial_insurance',
       'INV_industry_per_health_and_beauty', 'INV_industry_per_manufacture',
       'INV_industry_per_other', 'INV_industry_per_professional_services',
       'INV_industry_per_retail_service', 'INV_industry_per_transportation',
       'INV_industry_per_wholesale',
       'APP_participate_count', 'APP_bnnr_count', 'APP_game_count',
       'APP_bnnr_per', 'APP_game_per',
       'TXN_total_count', 'TXN_earn_count', 'TXN_redeem_count',
       'TXN_earn_count_per', 'TXN_redeem_count_per', 'TXN_total_point',
       'TXN_earn_point', 'TXN_redeem_point', 'TXN_earn_point_per',
       'TXN_redeem_point_per', 'TXN_industry_count', 'TXN_industry_per_retail',
       'TXN_industry_per_other', 'TXN_industry_per_dingding',
       'TXN_industry_per_financial', 'TXN_industry_per_digit',
       'TXN_industry_per_transportation', 'TXN_industry_per_leisure',
       'TXN_industry_per_health', 'TXN_industry_per_professional',
       'TXN_industry_per_food', 'AD_TAG', 'Gender_Code', 'ZIP_Code']].fillna(0)

In [18]:
df[['INV_time_mode', 'APP_time_mode', 'TXN_time_mode', 'TXN_zip_code']] = df[['INV_time_mode', 'APP_time_mode', 'TXN_time_mode', 'TXN_zip_code']].fillna('unknown')

In [20]:
df[['INV_time_avg_period', 'APP_time_avg_period', 'TXN_time_avg_period']] = df[['INV_time_avg_period', 'APP_time_avg_period', 'TXN_time_avg_period']].fillna(5259600)

In [23]:
df.isna().sum().sum()

0

In [24]:
df.to_csv('../feature_data/HappyGo_NOV_DEC_0509_FV.csv', index=False)